In [424]:
import pandas as pd
import numpy as np

In [425]:
df = pd.read_csv('fundacurrentlistings20240517_dataset.csv')

In [426]:
df = df.drop_duplicates()
df = df.drop(columns={'size', 'url', 'descrip', 'city', 'last_ask_price', 'insulation', 'heating', 'num_of_bathrooms'})
df = df.rename(columns={'ownership':'outside_space', 'building_type':'newbuild'})

# clean the price
df['price'] = df['price'].str.replace('€ ', '')
df['price'] = df['price'].str.replace(' /mnd', '')
df['price'] = df['price'].str.replace('k', '')
df['price'] = df['price'].str.replace('na', '0')
df['price'] = df['price'].str.replace('Huurprijs op aanvraag', '0')
df['price'] = df['price'].str.replace('.', '')

# drop all rows where dependent variable price is equal to zero and higher than 12500 euros a months. Those are buy listings listed wrongly
df = df.drop(df[df['price'] == '0'].index)
df = df.drop(df[pd.to_numeric(df['price']) > 12500].index)
df = df.drop(df[pd.to_numeric(df['price']) < 500].index)


# clean zip_code. Should be len = 6 with 4 numbers and 2 letters
df['zip_code'] = df['zip_code'].str[:7]
df['zip_code'] = df['zip_code'].str.replace(' ', '')

# remove non-houses
df = df.drop(df[df['kind_of_house'] == 'Garage'].index)
df = df.drop(df[df['kind_of_house'] == 'Inpandige garage'].index)
df = df.drop(df[df['kind_of_house'] == 'parkeerkelder'].index)
df = df.drop(df[df['kind_of_house'] == 'Parkeerplaats'].index)

# clean the year. We changed 'Voor XXXX', 'Na XXXX' and 'XXXX-YYYY' to 'XXXX'
df['year'] = df['year'].str.replace('Voor ', '')
df['year'] = df['year'].str.replace('Na ', '')
df['year'] = df['year'].str[:4]

def check_year_validity(label):
    if 0 < label < 2025:
        return label
    else:
        return np.nan

df['year'] = df['year'].astype(int).apply(check_year_validity)

def check_living_area_validity(label):
    if label != 'na':
        return label
    else:
        return np.nan

# clean living_area
df['living_area'] = df['living_area'].str.replace(' m²', '')
df['living_area'] = df['living_area'].astype(str).apply(check_living_area_validity)

df.describe()

,year
count,887.000000
mean,1959.910936
std,91.418085
min,1005.000000
25%,1906.000000
50%,1998.000000
75%,2024.000000
max,2024.000000


In [427]:
df.head(2)

,price,address,zip_code,year,living_area,kind_of_house,newbuild,num_of_rooms,layout,energy_label,outside_space,parking
0,1045,Schipluidenlaan 254,1062HE,2024,44,Portiekflat,Nieuwbouw,2 kamers (1 slaapkamer),Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,na,na,na
1,1068,Krijn Taconiskade 299,1087HW,2022,44,Portiekflat (appartement met open portiek),Nieuwbouw,1 kamer (1 slaapkamer),Aantal kamers1 kamer (1 slaapkamer)Aantal woon...,A++,na,na


In [428]:
# extract number of rooms in total
df['num_of_rooms'] = df['layout'].str.extract(r'Aantal kamers(\d+)')
# extract the number of bathrooms
df['num_of_bedrooms'] = df['layout'].str.extract(r'(\d+) slaapkamer')
# extract the number of bedrooms
df['num_of_bathrooms'] = df['layout'].str.extract(r'(\d+) badkamer')
# extract the number of seperate toilets
df['num_of_separate_toilets'] = df['layout'].str.extract(r'(\d+) apart')

# extract the number of floors
# df['total_num_of_floors'] = df['layout'].str.extract(r'Aantal woonlagen(\d+)') - drop this cause this is not correct in alot of the entry, so I'll delete it

# extract the floor number of house
df['floor_of_house'] = df['layout'].str.extract(r'Gelegen op(\w+)')
df['floor_of_house'] = df['floor_of_house'].str.replace('e', '')


# parking
df['parking'] = df['parking'].apply(lambda x: 1 if 'Soort parkeergelegenheid' in x else 0)

# dummy newbuild
df['newbuild'] = df['newbuild'].apply(lambda x: 1 if 'Nieuwbouw' in x else 0)

# Turn the energy labels into ordinal values with NaN if value is missing
def check_energy_label(label):
    if 'A++++' in label:
        return 8
    elif 'A+++' in label:
        return 7
    elif 'A++' in label:
        return 6
    elif 'A+' in label:
        return 5
    elif 'A' in label:
        return 4
    elif 'B' in label:
        return 3
    elif 'C' in label:
        return 2
    elif 'D' in label:
        return 1
    else:
        return np.nan

df['energy_label'] = df['energy_label'].astype(str).apply(check_energy_label)

df.head(20)

,price,address,zip_code,year,living_area,kind_of_house,newbuild,num_of_rooms,layout,energy_label,outside_space,parking,num_of_bedrooms,num_of_bathrooms,num_of_separate_toilets,floor_of_house
0,1045,Schipluidenlaan 254,1062HE,2024,44,Portiekflat,1,2,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,NaN,na,0,1,NaN,NaN,1
1,1068,Krijn Taconiskade 299,1087HW,2022,44,Portiekflat (appartement met open portiek),1,1,Aantal kamers1 kamer (1 slaapkamer)Aantal woon...,6.0,na,0,1,NaN,NaN,3
2,1099,Haarlemmerweg,1014BL,2024,50,Galerijflat,1,2,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,NaN,Nee,0,1,NaN,NaN,NaN
3,1103,Krijn Taconiskade 441,1087HW,2022,44,Portiekflat (appartement met open portiek),1,1,Aantal kamers1 kamer (1 slaapkamer)Aantal woon...,6.0,na,0,1,NaN,NaN,5
4,1105,Willem Frogerstraat 41,1062HZ,2024,56,Portiekflat,1,2,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,NaN,na,0,1,NaN,NaN,1
5,1115,Schipluidenlaan 270,1062HE,2024,50,Portiekflat,1,2,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,NaN,na,0,1,NaN,NaN,1
6,1120,Staalmeesterslaan 377,1057PG,1971,58,Galerijflat (appartement),0,2,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,5.0,na,0,1,NaN,NaN,13
7,1120,Staalmeesterslaan 380,1057PG,1971,58,Galerijflat (appartement),0,2,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,5.0,na,0,1,NaN,NaN,14
8,1130,Olga de Haasstraat 513,1095PG,2024,58,Portiekflat,1,2,Aantal kamers2 kamers (1 slaapkamer)Aantal woo...,NaN,na,0,1,NaN,NaN,28
9,1135,Willem Frogerstraat 53,1062HZ,2024,80,Portiekflat,1,4,Aantal kamers4 kamers (3 slaapkamers)Aantal wo...,NaN,na,0,3,NaN,NaN,1


In [429]:
df.to_csv('v7.csv')
# df.sort_values('total_num_of_floors', ascending=False, key=pd.to_numeric)

In [430]:
# df['kind_of_house'] = df['kind_of_house'].str.casefold()
# df['kind_of_house'].unique().tolist()
